In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
tfpd = tfp.distributions
import pandas as pd
import numpy as np
import pybullet as p
import time
import pybullet_data
import jsonpickle
import os
from grippers import Suction
import threading

######

opo = [((0.4999997471314968, 1.2956212843440064e-06, 0.014989813953784152),
  (7.3716523356465255e-06,
   -2.278102253914087e-06,
   8.779366957142966e-06,
   0.9999999999316959)),
 ((0.49999992475663513, 0.07000112902035814, 0.014989489968807479),
  (1.9108787251728214e-05,
   -4.303750121310861e-06,
   1.6186749115449988e-06,
   0.9999999998068559)),
 ((0.499999776084307, 0.13999996533389208, 0.014989687073218719),
  (8.850380280004497e-06,
   -7.230234022126371e-06,
   8.460564891452243e-07,
   0.9999999999343393)),
 ((0.49999984815002413, 0.20999994852193918, 0.014989860492678008),
  (2.6815242043856674e-06,
   -5.253120602529134e-06,
   -2.2351298040415624e-07,
   0.999999999982582)),
 ((0.5000041979739713, 0.035000413980079005, 0.04247946418220433),
  (0.707099689533288,
   2.329368796799023e-05,
   8.748219862131678e-06,
   0.7071138723309002)),
 ((0.5000023948655364, 0.10499945582550495, 0.04247937315026681),
  (0.707105162660926,
   1.3188804904686976e-05,
   -3.919509284554507e-06,
   0.7071083995746043)),
 ((0.5000008650676556, 0.17500042563751045, 0.0424761229416361),
  (0.7070920678471995,
   2.708401531351398e-05,
   -2.764430689092011e-05,
   0.7071214931607015))]

######

ASSETS_ROOT = os.path.abspath("urdf")
obj_ids = {'fixed': [], 'rigid': [], 'deformable': []}
homej = np.array([-1, -0.5, 0.5, -0.5, -0.5, 0]) * np.pi

physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0,0,-9.81)

target = (-0.07796166092157364, 0.005451506469398737, -0.06238798052072525)
dist = 1.0
yaw = 89.6000747680664
pitch = -17.800016403198242
p.resetDebugVisualizerCamera(dist,yaw,pitch,target)

planeId = p.loadURDF(os.path.join(ASSETS_ROOT, 'plane/plane.urdf'))

origin = np.zeros(3)
noRotation = p.getQuaternionFromEuler([0,0,0])

ur5 = p.loadURDF(os.path.join(ASSETS_ROOT, 'ur5/ur5.urdf'), origin, noRotation, 
                useFixedBase=True)
suc = Suction(ASSETS_ROOT, ur5, 9, obj_ids)
n_joints = p.getNumJoints(ur5)
joints = [p.getJointInfo(ur5, i) for i in range(n_joints)]
joints = [j[0] for j in joints if j[2] == p.JOINT_REVOLUTE]

for j, hj in zip(joints, homej):
    p.resetJointState(ur5, j, hj)

# only static dimension shapes
shapes = ['cube', 'cuboid', 'thin_cuboid', 'thinner_cuboid']
valid_rots = {
    'cube': ([0], [0], [0]),
    'cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thin_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
    'thinner_cuboid': ([0, np.pi/2], [0, np.pi/2], [0, np.pi/2]),
}

objPos = np.array([0.5,0,0.1])

def load_objr(otype, od=[0,0,0], rot=[0,0,0], tp=True):
    
    rot = p.getQuaternionFromEuler(rot)
    
    ret = p.loadURDF(os.path.join(ASSETS_ROOT, f'objects/{otype}.urdf'), objPos+np.array(od), rot)
    
    if tp:
        for _ in range(100):
            p.stepSimulation()
            time.sleep(1./240.)
        
    return ret

def load_obj(otype, od=[0,0,0], rot=[0,0,0], tp=True):
    ret = p.loadURDF(os.path.join(ASSETS_ROOT, f'objects/{otype}.urdf'), np.array(od), rot)
        
    return ret

def load_objs(otlist, opo, oft=[0,0,0]):
    
    oft = np.array(oft)
    
    for i, (ot, (pos, ori)) in enumerate(zip(otlist, opo)):
        pert = np.zeros(3)
        pert[:2] = np.random.uniform(-.007,0.007,2)
        print(pert)
        time.sleep(0.5)
        
        load_obj(ot, pos+pert+oft, ori)

# towards, right, up
# rots:
# rol head anti
# rol into/out of up/down
# rol into/out of left/right

objs = []

time.sleep(5)
np.random.seed(40)

load_objs(['cube', 'cube', 'cube', 'cube', 'cuboid', 'cuboid', 'cuboid'], opo)

# objs = []

# objs.append(load_objr('cube'))
# objs.append(load_objr('cube', [0,0.07,0]))
# objs.append(load_objr('cube', [0,0.14,0]))
# objs.append(load_objr('cube', [0,0.21,0]))

# objs.append(load_objr('cuboid', [0,0.035,0], rot=[np.pi/2,0,0]))
# objs.append(load_objr('cuboid', [0,0.14-.035,0], rot=[np.pi/2,0,0]))
# objs.append(load_objr('cuboid', [0,0.21-.035,0], rot=[np.pi/2,0,0]))


# opo = []
# for i, ob in enumerate(objs):
#     opo.append(p.getBasePositionAndOrientation(ob))
# print()

# load_objs(otlist, opo)
input()

p.disconnect()

pybullet build time: Jul 21 2022 19:48:53


[-0.00129238 -0.00622488  0.        ]
[ 0.00403949 -0.00297773  0.        ]
[-0.00069509 -0.00274523  0.        ]
[0.00036959 0.00173337 0.        ]
[0.00387486 0.00260738 0.        ]
[0.00673314 0.00141143 0.        ]
[0.00439556 0.00292103 0.        ]


In [5]:
opo

[((0.4999997471314968, 1.2956212843440064e-06, 0.014989813953784152),
  (7.3716523356465255e-06,
   -2.278102253914087e-06,
   8.779366957142966e-06,
   0.9999999999316959)),
 ((0.49999992475663513, 0.07000112902035814, 0.014989489968807479),
  (1.9108787251728214e-05,
   -4.303750121310861e-06,
   1.6186749115449988e-06,
   0.9999999998068559)),
 ((0.499999776084307, 0.13999996533389208, 0.014989687073218719),
  (8.850380280004497e-06,
   -7.230234022126371e-06,
   8.460564891452243e-07,
   0.9999999999343393)),
 ((0.49999984815002413, 0.20999994852193918, 0.014989860492678008),
  (2.6815242043856674e-06,
   -5.253120602529134e-06,
   -2.2351298040415624e-07,
   0.999999999982582)),
 ((0.5000041979739713, 0.035000413980079005, 0.04247946418220433),
  (0.707099689533288,
   2.329368796799023e-05,
   8.748219862131678e-06,
   0.7071138723309002)),
 ((0.5000023948655364, 0.10499945582550495, 0.04247937315026681),
  (0.707105162660926,
   1.3188804904686976e-05,
   -3.919509284554507e-06,